In [8]:
!pip install datasets
!pip install pandas requests onedrivedownloader


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import requests
import io
import pandas as pd

# Import the download function instead of OneDriveDownloader class
from onedrivedownloader import download

# Main dataset is kept here
onedrive_link = "https://indianinstituteofscience-my.sharepoint.com/:u:/g/personal/rishavg_iisc_ac_in/Ecr2JHgh-2VBg3bYhcqFQKYBcpNLNsLEPfGNLAKfdKckfQ"

# Extract the file ID from the OneDrive link - not needed for current download method
file_id = onedrive_link.split('/')[-1]  # Assuming the file ID is the last part of the URL

# Download the file using the download function
# filename should be provided and the file will be automatically saved, you can directly input your local path
download(onedrive_link, filename="all_users_rating.tsv")

'all_users_rating.tsv'

In [3]:
# Read the downloaded file into a DataFrame
import pandas as pd

all_users_rating_df = pd.read_csv("all_users_rating.tsv", sep='\t', on_bad_lines='skip')

In [4]:
all_users_rating_df.shape

(408873, 15)

Handling missing values:
*   Identify missing values
*   Remove missing values


In [5]:
all_users_rating_df.isnull().sum()

marketplace           0
customer_id           0
review_id             0
product_id            0
product_parent        0
product_title         0
product_category      0
star_rating           1
helpful_votes         1
total_votes           1
vine                  1
verified_purchase     1
review_headline      23
review_body           1
review_date          56
dtype: int64

In [6]:
# Drop Rows with Any Missing Values
all_users_rating_df.dropna(inplace=True)

all_users_rating_df.shape

(408795, 15)

In [7]:
# Drop unnecessary columns
columns_to_drop = ['vine', 'marketplace', 'verified_purchase', 'product_parent', 'review_body', 'product_category']
all_users_rating_df = all_users_rating_df.drop(columns=columns_to_drop, errors='ignore')

In [8]:
all_users_rating_df.columns

Index(['customer_id', 'review_id', 'product_id', 'product_title',
       'star_rating', 'helpful_votes', 'total_votes', 'review_headline',
       'review_date'],
      dtype='object')

In [9]:
# Distinct No. of Product Title
distinct_product_count = all_users_rating_df['product_title'].nunique()
print("Distinct No. of Product", distinct_product_count)

# Distinct No. of Customers
distinct_customer_count = all_users_rating_df['customer_id'].nunique()
print("Distinct No. of Customers", distinct_customer_count)

Distinct No. of Product 177618
Distinct No. of Customers 232659


In [10]:
# Create an df with all the customers having rating count >= 3

# Group by customer_id and count the number of ratings for each customer
customer_rating_counts = all_users_rating_df.groupby('customer_id')['star_rating'].count()

# Filter the customer_rating_counts to include only customers with 3 or more ratings
customers_with_at_least_3_ratings = customer_rating_counts[customer_rating_counts >= 3].index

# Create a new DataFrame containing only the rows where the customer_id is in the customers_with_at_least_3_ratings list
df_filtered_user_rating = all_users_rating_df[all_users_rating_df['customer_id'].isin(customers_with_at_least_3_ratings)]

df_filtered_user_rating.head()

,customer_id,review_id,product_id,product_title,star_rating,helpful_votes,total_votes,review_headline,review_date
3,50732546,RATOTLA3OF70O,0373836635,Colby Conspiracy (Colby Agency),5.0,2.0,2.0,fine author on her A-game,2005-10-14
6,53000124,R1KJ6MB7MRSQFF,0805076069,Bait and Switch: The (Futile) Pursuit of the A...,4.0,9.0,11.0,I viewed this the opposite of Publisher's Week...,2005-10-14
12,27925116,R7M06Z88PD7SX,0029148510,Acts of War: Behavior of Men in Battle,4.0,14.0,14.0,Solid Book About Facing Death and Killing in War,2005-10-14
20,50732546,R17026W0TBFZGL,045121692X,"The Last Heiress (Friarsgate Inheritance, Book 4)",5.0,1.0,2.0,superb historical romance,2005-10-14
32,41549714,R2W8C8DO33WTMP,0312853165,"The Demon Princes, Vol. 2: The Face * The Book...",4.0,6.0,7.0,Fourth and Fifth Books of the Demon Princes Se...,2005-10-14


In [42]:
customers_with_at_least_3_ratings.shape

(20291,)

In [11]:
df_filtered_user_rating.shape

(172430, 9)

In [12]:
# Remove the review with no helpful upvote
df_filtered_user_rating = df_filtered_user_rating[df_filtered_user_rating['helpful_votes'] < 1]

In [13]:
df_filtered_user_rating.shape

(17654, 9)

In [14]:
# Distinct No. of Product(Book) after filtering
distinct_product_count = df_filtered_user_rating['product_title'].nunique()
print("Distinct no. of Product", distinct_product_count)

# Distinct No. of Customers after filtering
distinct_customer_count = df_filtered_user_rating['customer_id'].nunique()
print("Distinct no. of Customers", distinct_customer_count)

# Distinct No. of reviews after filtering
distinct_review_count = df_filtered_user_rating['review_id'].nunique()
print("Distinct no. of reviews", distinct_review_count)

Distinct no. of Product 13676
Distinct no. of Customers 7159
Distinct no. of reviews 17654


In [15]:
df_filtered_books = df_filtered_user_rating[['product_id', 'product_title']].drop_duplicates()
print('Shape of df_filtered_books: ', df_filtered_books.shape)
df_filtered_books.head()

Shape of df_filtered_books:  (13944, 2)


,product_id,product_title
49,0373513194,Kiss of the Blue Dragon (Silhouette Bombshell)
189,1410202984,Dahcotah: Life and Legends of the Sioux
220,0816524718,Navajo Nation Peacemaking: Living Traditional ...
255,1591160529,"Inuyasha, Volume 5"
277,1571781838,Love Made Visible


In [31]:
import io
#from google.colab import files # Importing the necessary module

# Convert the DataFrame to CSV format and save it to a file
# Saving the dataframe to a csv file first
df_filtered_user_rating.to_csv('filtered_user_rating.csv', index=False)

# Download the file to local system - needed 03-EDA-Step
#files.download('filtered_user_rating.csv')

In [43]:
import io
#from google.colab import files # Importing the necessary module

# Convert the DataFrame to CSV format and save it to a file
# Saving the dataframe to a csv file first
df_filtered_books.to_csv('filtered_books_data.csv', index=False)

# Download the file to local system - needed 03-EDA-Step
#files.download('filtered_books_data.csv')

Download book metadata from source

In [44]:
from onedrivedownloader import download

# Main dataset is kept here
onedrive_link = "https://indianinstituteofscience-my.sharepoint.com/:x:/g/personal/rishavg_iisc_ac_in/EdVMAFu4QJxAnyv3U4Bryx0BGPQT1ZzDKe4CftC6NmhnBQ?e=elTH2h"

# Extract the file ID from the OneDrive link - not needed for current download method
file_id = onedrive_link.split('/')[-1]  # Assuming the file ID is the last part of the URL

# Download the file using the download function
# filename should be provided and the file will be automatically saved, you can directly input your local path
download(onedrive_link, filename="Books_metadata.csv")

'Books_metadata.csv'

In [19]:
import pandas as pd

# Assuming 'Books_Data_Clean.csv' is in the current working directory
books_metadata_df = pd.read_csv('Books_metadata.csv')

# Print some info about the DataFrame
books_metadata_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1070 entries, 0 to 1069
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index                1070 non-null   int64  
 1   Publishing Year      1069 non-null   float64
 2   Book Name            1047 non-null   object 
 3   Author               1070 non-null   object 
 4   language_code        1017 non-null   object 
 5   Author_Rating        1070 non-null   object 
 6   Book_average_rating  1070 non-null   float64
 7   Book_ratings_count   1070 non-null   int64  
 8   genre                1070 non-null   object 
 9   gross sales          1070 non-null   float64
 10  publisher revenue    1070 non-null   float64
 11  sale price           1070 non-null   float64
 12  sales rank           1070 non-null   int64  
 13  Publisher            1070 non-null   object 
 14  units sold           1070 non-null   int64  
dtypes: float64(5), int64(4), object(6)
mem

In [20]:
print('Shape of books_metadata_df: ', books_metadata_df.shape)

Shape of books_metadata_df:  (1070, 15)


In [21]:
books_metadata_df.head()

,index,Publishing Year,Book Name,Author,language_code,Author_Rating,Book_average_rating,Book_ratings_count,genre,gross sales,publisher revenue,sale price,sales rank,Publisher,units sold
0,0,1975.0,Beowulf,"Unknown, Seamus Heaney",en-US,Novice,3.42,155903,genre fiction,34160.0,20496.0,4.88,1,HarperCollins Publishers,7000
1,1,1987.0,Batman: Year One,"Frank Miller, David Mazzucchelli, Richmond Lew...",eng,Intermediate,4.23,145267,genre fiction,12437.5,7462.5,1.99,2,HarperCollins Publishers,6250
2,2,2015.0,Go Set a Watchman,Harper Lee,eng,Novice,3.31,138669,genre fiction,47795.0,28677.0,8.69,3,"Amazon Digital Services, Inc.",5500
3,3,2008.0,When You Are Engulfed in Flames,David Sedaris,en-US,Intermediate,4.04,150898,fiction,41250.0,24750.0,7.50,3,Hachette Book Group,5500
4,4,2011.0,Daughter of Smoke & Bone,Laini Taylor,eng,Intermediate,4.04,198283,genre fiction,37952.5,22771.5,7.99,4,Penguin Group (USA) LLC,4750


In [22]:
# Merge books_entity_df and book_df based on 'Book Name'
merged_user_rating_book_df = pd.merge(books_metadata_df, df_filtered_user_rating, left_on='Book Name', right_on='product_title', how='inner')

print('Shape of merged_user_rating_book_df: ', merged_user_rating_book_df.shape)

Shape of merged_user_rating_book_df:  (351, 24)


In [23]:
# Print the merged DataFrame
merged_user_rating_book_df.head()

,index,Publishing Year,Book Name,Author,language_code,Author_Rating,Book_average_rating,Book_ratings_count,genre,gross sales,...,units sold,customer_id,review_id,product_id,product_title,star_rating,helpful_votes,total_votes,review_headline,review_date
0,21,2000.0,The Rescue,Nicholas Sparks,en-US,Intermediate,4.10,142092,genre fiction,8569.34,...,2866,40817134,R1UJ7781YB4INE,0446610399,The Rescue,4.0,0.0,0.0,Loved it! Much better than I had anticipated,2005-08-24
1,21,2000.0,The Rescue,Nicholas Sparks,en-US,Intermediate,4.10,142092,genre fiction,8569.34,...,2866,14818676,R3LHUCZXAKE40D,0446610399,The Rescue,4.0,0.0,2.0,This was a pretty good book,2005-01-21
2,26,1993.0,The Virgin Suicides,Jeffrey Eugenides,eng,Intermediate,3.83,159249,genre fiction,26904.06,...,2466,13920601,R2GYAKQFEOF3L5,0446670251,The Virgin Suicides,4.0,0.0,2.0,"Disturbing, yet mysterious",2005-09-02
3,26,1993.0,The Virgin Suicides,Jeffrey Eugenides,eng,Intermediate,3.83,159249,genre fiction,26904.06,...,2466,51244189,R2LWLH1O0RECZN,0446670251,The Virgin Suicides,5.0,0.0,0.0,Well-written from an interesting perspective,2005-04-19
4,26,1993.0,The Virgin Suicides,Jeffrey Eugenides,eng,Intermediate,3.83,159249,genre fiction,26904.06,...,2466,52473350,R1I5PAXUMD1APY,0446670251,The Virgin Suicides,4.0,0.0,0.0,The Blacker Side of Americana,2005-03-07


In [45]:
# Distinct No. of Product(Book) after filtering
distinct_product_count = merged_user_rating_book_df['product_title'].nunique()
print("Distinct No. of Product", distinct_product_count)

# Distinct No. of Customers after filtering
distinct_customer_count = merged_user_rating_book_df['customer_id'].nunique()
print("Distinct No. of Customers", distinct_customer_count)

Distinct No. of Product 134
Distinct No. of Customers 326
